# Schottky Barrier Diode

## Metal-Semiconductor Junction Physics

Schottky diodes are formed at metal-semiconductor interfaces and offer advantages over PN junctions including faster switching and lower forward voltage drop.

**Learning Objectives:**
- Understand Schottky barrier formation
- Compare with PN junction diodes
- Analyze barrier height effects
- Simulate I-V characteristics

In [ ]:
# Setup: Load PADRE environment (required on nanoHUB)
# This cell loads the PADRE simulator into your environment.
# If running locally with PADRE already in your PATH, this will be skipped gracefully.

from nanohubpadre import use

# Load the PADRE simulator environment
%use padre-2.4E-r15

print("PADRE environment setup complete.")

---

## 1. Schottky Barrier Physics

### 1.1 Band Diagram

When a metal contacts an n-type semiconductor:

```
    Metal     |     N-type Semiconductor
              |
    ----      |     ----
    Ef_m      |          \____ Ec
              |                \
              |                 \---- Ef_s
              |                       
              |              \_______ Ev
              |
         <-- Depletion -->
```

### 1.2 Key Parameters

- **Schottky barrier height** ($\phi_B$): Energy barrier at the interface
  $$\phi_B = \phi_m - \chi$$
  where $\phi_m$ is metal workfunction and $\chi$ is semiconductor electron affinity

- **Built-in potential**:
  $$V_{bi} = \phi_B - \frac{kT}{q}\ln\left(\frac{N_c}{N_d}\right)$$

- **Thermionic emission current**:
  $$J = A^* T^2 \exp\left(-\frac{q\phi_B}{kT}\right)\left[\exp\left(\frac{qV}{kT}\right) - 1\right]$$
  where $A^* \approx 120$ A/cm$^2$/K$^2$ is Richardson's constant

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from nanohubpadre import create_schottky_diode

# Physical constants
q = 1.6e-19       # Elementary charge (C)
kT = 0.0259       # Thermal voltage at 300K (eV)
chi_si = 4.05     # Silicon electron affinity (eV)
Nc = 2.8e19       # Effective density of states in CB (cm^-3)
A_star = 120      # Richardson constant (A/cm^2/K^2)
T = 300           # Temperature (K)

# Metal workfunctions (eV)
metals = {
    'Ti': 4.33,
    'Cr': 4.50,
    'W': 4.55,
    'Mo': 4.60,
    'Au': 5.10,
    'Pt': 5.65
}

print("Schottky Barrier Heights on n-Si:")
print("="*40)
for metal, phi_m in metals.items():
    phi_B = phi_m - chi_si
    print(f"{metal}: phi_m = {phi_m:.2f} eV, phi_B = {phi_B:.2f} eV")

---

## 2. Creating a Schottky Diode

Let's create Schottky diodes with different barrier heights.

In [ ]:
# Create a basic Schottky diode (Ti contact on n-Si)
sim_ti = create_schottky_diode(
    # Geometry
    length=2.0,          # Device length (um)
    width=1.0,           # Device width (um)
    
    # Doping
    doping=1e17,         # N-type doping (cm^-3)
    doping_type='n',     # N-type substrate
    
    # Contact
    workfunction=4.33,   # Ti workfunction (eV)
    
    # Output
    log_bands_eq=True,
    log_iv=True,
    iv_file="ti_schottky",
    
    # Voltage sweep
    forward_sweep=(0.0, 0.5, 0.02)
)

print("Schottky Diode Configuration")
print("="*40)
print(f"Metal: Ti (phi_m = 4.33 eV)")
print(f"Barrier height: {4.33 - chi_si:.2f} eV")
print(f"Substrate: n-Si (Nd = 1e17 cm^-3)")

# Run the simulation
print("\nRunning equilibrium simulation...")
result = sim_ti.run()
if result.returncode == 0:
    print("Simulation completed successfully!")
else:
    print(f"Simulation failed with return code: {result.returncode}")

In [ ]:
# View the generated deck
print("PADRE Input Deck:")
print("="*60)
print(sim_ti.generate_deck())

# Plot equilibrium band diagram
print("\n" + "="*60)
print("Equilibrium Band Diagram:")
sim_ti.plot_band_diagram(title="Ti-Si Schottky Diode - Equilibrium")

---

## 3. Barrier Height Effect

The Schottky barrier height strongly affects the I-V characteristics.

In [ ]:
# Create diodes with different barrier heights and run simulations
barrier_heights = [0.3, 0.5, 0.7]  # eV (reduced to 3 for faster simulation)
workfunctions = [chi_si + phi_B for phi_B in barrier_heights]

print("Comparing Different Barrier Heights:")
print("="*50)

simulations = {}
for phi_B, phi_m in zip(barrier_heights, workfunctions):
    sim = create_schottky_diode(
        doping=1e17,
        workfunction=phi_m,
        log_iv=True,
        forward_sweep=(0.0, 0.5, 0.02)
    )
    
    # Calculate theoretical saturation current density
    Js = A_star * T**2 * np.exp(-phi_B / kT)
    print(f"phi_B = {phi_B:.1f} eV: Js = {Js:.2e} A/cm^2")
    
    # Run simulation
    print(f"  Running simulation...")
    result = sim.run()
    if result.returncode == 0:
        print(f"  Simulation completed!")
        simulations[phi_B] = sim
    else:
        print(f"  Simulation failed!")

In [ ]:
# Plot I-V curves from PADRE simulations vs theoretical
V_theory = np.linspace(0, 0.5, 100)
A = 1e-8  # Device area (cm^2)

plt.figure(figsize=(12, 5))

# Linear scale
plt.subplot(1, 2, 1)
colors = ['blue', 'orange', 'green']
for i, phi_B in enumerate(barrier_heights):
    # Theoretical curve
    Js = A_star * T**2 * np.exp(-phi_B / kT)
    I_theory = A * Js * (np.exp(V_theory / kT) - 1)
    plt.plot(V_theory, I_theory * 1e6, '--', color=colors[i], alpha=0.5, 
             label=f'Theory $\phi_B$ = {phi_B} eV')
    
    # Simulated curve
    if phi_B in simulations:
        iv_data = simulations[phi_B].get_iv_data()
        V_sim, I_sim = iv_data.get_iv_data(electrode=2)
        plt.plot(V_sim, np.abs(I_sim) * 1e6, '-', color=colors[i], linewidth=2,
                label=f'PADRE $\phi_B$ = {phi_B} eV')

plt.xlabel('Voltage (V)', fontsize=12)
plt.ylabel('Current (µA)', fontsize=12)
plt.title('Schottky I-V: Effect of Barrier Height', fontsize=14)
plt.legend(fontsize=8)
plt.grid(True, alpha=0.3)

# Log scale
plt.subplot(1, 2, 2)
for i, phi_B in enumerate(barrier_heights):
    # Theoretical curve
    Js = A_star * T**2 * np.exp(-phi_B / kT)
    I_theory = A * Js * (np.exp(V_theory / kT) - 1)
    plt.semilogy(V_theory, np.abs(I_theory), '--', color=colors[i], alpha=0.5)
    
    # Simulated curve
    if phi_B in simulations:
        iv_data = simulations[phi_B].get_iv_data()
        V_sim, I_sim = iv_data.get_iv_data(electrode=2)
        mask = np.abs(I_sim) > 1e-20
        plt.semilogy(V_sim[mask], np.abs(I_sim[mask]), '-', color=colors[i], linewidth=2)

plt.xlabel('Voltage (V)', fontsize=12)
plt.ylabel('Current (A)', fontsize=12)
plt.title('Schottky I-V (Log Scale)', fontsize=14)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Extract parameters from simulations
print("\nParameter Extraction from PADRE Simulations:")
print("="*60)
for phi_B in barrier_heights:
    if phi_B in simulations:
        iv_data = simulations[phi_B].get_iv_data()
        V_sim, I_sim = iv_data.get_iv_data(electrode=2)
        
        # Extract ideality factor from log(I) vs V slope
        mask = (V_sim > 0.1) & (V_sim < 0.3) & (np.abs(I_sim) > 1e-20)
        if np.sum(mask) > 2:
            log_I = np.log(np.abs(I_sim[mask]))
            V_fit = V_sim[mask]
            slope, intercept = np.polyfit(V_fit, log_I, 1)
            n_sim = 1 / (slope * kT)  # Ideality factor
            Is_sim = np.exp(intercept)  # Saturation current
            print(f"phi_B = {phi_B:.1f} eV:")
            print(f"  Simulated ideality factor: n = {n_sim:.2f}")
            print(f"  Simulated saturation current: Is = {Is_sim:.2e} A")

---

## 4. Schottky vs PN Junction

Let's compare Schottky and PN junction diodes.

In [ ]:
from nanohubpadre import create_pn_diode

# Schottky diode
sim_schottky = create_schottky_diode(
    doping=1e17,
    workfunction=4.55,  # phi_B ~ 0.5 eV
    log_iv=True,
    forward_sweep=(0.0, 0.8, 0.02)
)

# PN junction diode
sim_pn = create_pn_diode(
    p_doping=1e17,
    n_doping=1e17,
    log_iv=True,
    forward_sweep=(0.0, 0.8, 0.02)
)

print("Running Schottky and PN Junction Simulations...")
print("="*50)

print("\nSchottky diode simulation...")
result_schottky = sim_schottky.run()
if result_schottky.returncode == 0:
    print("  Completed!")
    
print("\nPN junction simulation...")
result_pn = sim_pn.run()
if result_pn.returncode == 0:
    print("  Completed!")

print("\nComparison: Schottky vs PN Junction")
print("="*50)
print("\nSchottky Diode Advantages:")
print("  - Lower turn-on voltage (~0.3V vs ~0.7V)")
print("  - Faster switching (majority carrier device)")
print("  - No minority carrier storage")
print("\nSchottky Diode Disadvantages:")
print("  - Higher reverse leakage current")
print("  - Lower breakdown voltage")

In [ ]:
# Plot PADRE simulation comparison: Schottky vs PN Junction
plt.figure(figsize=(12, 5))

# Get simulation data
iv_schottky = sim_schottky.get_iv_data()
V_schottky, I_schottky = iv_schottky.get_iv_data(electrode=2)

iv_pn = sim_pn.get_iv_data()
V_pn, I_pn = iv_pn.get_iv_data(electrode=2)

# Linear scale
plt.subplot(1, 2, 1)
plt.plot(V_schottky, np.abs(I_schottky) * 1e6, 'b-', linewidth=2, label='Schottky (PADRE)')
plt.plot(V_pn, np.abs(I_pn) * 1e6, 'r-', linewidth=2, label='PN Junction (PADRE)')
plt.xlabel('Voltage (V)', fontsize=12)
plt.ylabel('Current (µA)', fontsize=12)
plt.title('Schottky vs PN Junction (Linear)', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.xlim(0, 0.8)

# Log scale
plt.subplot(1, 2, 2)
mask_schottky = np.abs(I_schottky) > 1e-20
mask_pn = np.abs(I_pn) > 1e-20
plt.semilogy(V_schottky[mask_schottky], np.abs(I_schottky[mask_schottky]), 'b-', linewidth=2, label='Schottky')
plt.semilogy(V_pn[mask_pn], np.abs(I_pn[mask_pn]), 'r-', linewidth=2, label='PN Junction')
plt.xlabel('Voltage (V)', fontsize=12)
plt.ylabel('Current (A)', fontsize=12)
plt.title('Schottky vs PN Junction (Log)', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.xlim(0, 0.8)

plt.tight_layout()
plt.show()

# Find turn-on voltages (at 1µA)
target_current = 1e-6  # 1 µA
V_on_schottky = np.interp(target_current, np.abs(I_schottky), V_schottky)
V_on_pn = np.interp(target_current, np.abs(I_pn), V_pn)

print(f"\nTurn-on voltages (at 1µA) from PADRE simulations:")
print(f"  Schottky: ~{V_on_schottky:.2f} V")
print(f"  PN Junction: ~{V_on_pn:.2f} V")
print(f"\nDifference: {V_on_pn - V_on_schottky:.2f} V lower for Schottky")

---

## 5. Band Diagram Analysis

Examining the band structure under different bias conditions.

In [ ]:
# Simulation with band diagram logging at different biases
sim_bands = create_schottky_diode(
    length=1.0,
    doping=1e17,
    workfunction=4.55,
    
    # Log bands at equilibrium and under bias
    log_bands_eq=True,
    log_bands_bias=True,
    
    # Forward and reverse sweeps
    forward_sweep=(0.0, 0.3, 0.1),
    reverse_sweep=(0.0, -2.0, -0.5)
)

print("Running band diagram simulation...")
result_bands = sim_bands.run()
if result_bands.returncode == 0:
    print("Simulation completed successfully!")
    print("\nOutput files generated:")
    print("  - cbeq, vbeq: Equilibrium bands")
    print("  - cbfwd, vbfwd: Forward bias bands")
    print("  - cbrev, vbrev: Reverse bias bands")
else:
    print(f"Simulation failed with return code: {result_bands.returncode}")

In [ ]:
# Plot band diagrams from PADRE simulation
print("Band Diagram from PADRE Simulation:")
sim_bands.plot_band_diagram(title="Schottky Diode - Equilibrium Band Diagram")

# Also plot theoretical band diagrams for comparison
def schottky_bands(x, phi_B, Vbi, V_applied, W):
    """Calculate Schottky diode band diagram"""
    Eg = 1.12  # Si bandgap
    Ec = np.zeros_like(x)
    Ev = np.zeros_like(x)
    
    V_eff = Vbi - V_applied
    W_eff = W * np.sqrt(max(V_eff/Vbi, 0.01))
    
    for i, xi in enumerate(x):
        if xi < W_eff:
            # Depletion region (parabolic)
            V_drop = V_eff * (1 - xi/W_eff)**2
            Ec[i] = phi_B - V_applied + V_drop - Eg/2
        else:
            # Neutral region
            Ec[i] = phi_B - Vbi - Eg/2
        Ev[i] = Ec[i] - Eg
    
    return Ec, Ev

# Theoretical comparison parameters
phi_B = 0.5
Nd = 1e17
eps_si = 11.7 * 8.85e-14
Vbi = phi_B - kT * np.log(Nc / Nd)
W0 = np.sqrt(2 * eps_si * Vbi / (q * Nd)) * 1e4  # um

x = np.linspace(0, 0.5, 200)
biases = [0, 0.2, -1.0, -2.0]

plt.figure(figsize=(12, 8))
plt.suptitle('Theoretical Schottky Diode Band Diagrams', fontsize=14)

for i, V in enumerate(biases):
    plt.subplot(2, 2, i+1)
    Ec, Ev = schottky_bands(x, phi_B, Vbi, V, W0)
    
    plt.plot(x, Ec, 'b-', linewidth=2, label='Ec')
    plt.plot(x, Ev, 'r-', linewidth=2, label='Ev')
    plt.axhline(y=0, color='g', linestyle='--', label='Ef (metal)')
    
    # Show barrier
    plt.annotate('', xy=(0, phi_B - V), xytext=(0, 0),
                arrowprops=dict(arrowstyle='<->', color='purple'))
    
    plt.xlabel('Position (µm)')
    plt.ylabel('Energy (eV)')
    plt.title(f'V = {V} V')
    plt.legend(loc='upper right', fontsize=8)
    plt.grid(True, alpha=0.3)
    plt.xlim(0, 0.5)

plt.tight_layout()
plt.show()

---

## 6. Barrier Lowering Effect

At high electric fields, the Schottky barrier is lowered (image force lowering).

In [ ]:
# Simulation with barrier lowering enabled
sim_barrier = create_schottky_diode(
    doping=1e17,
    workfunction=4.55,
    barrier_lowering=True,  # Enable image force lowering
    log_iv=True,
    forward_sweep=(0.0, 0.4, 0.02),
    reverse_sweep=(0.0, -5.0, -0.25)
)

print("Barrier lowering effect:")
print("  delta_phi = sqrt(qE / 4*pi*eps)")
print("  This reduces the effective barrier at high fields")

---

## 7. Complete Simulation Example

In [ ]:
# Complete Schottky diode characterization
sim_complete = create_schottky_diode(
    # Geometry
    length=2.0,
    width=1.0,
    
    # Mesh
    nx=200,
    ny=3,
    
    # Doping
    doping=1e17,
    doping_type='n',
    
    # Schottky contact
    workfunction=4.55,
    barrier_lowering=True,
    surf_rec=True,
    
    # Models
    temperature=300,
    srh=True,
    conmob=True,
    fldmob=True,
    
    # Output
    log_iv=True,
    iv_file="schottky_complete",
    log_bands_eq=True,
    log_bands_bias=True,
    
    # Sweeps
    forward_sweep=(0.0, 0.5, 0.01),
    reverse_sweep=(0.0, -5.0, -0.1)
)

print("Running Complete Schottky Diode Simulation")
print("="*50)
result_complete = sim_complete.run()
if result_complete.returncode == 0:
    print("Simulation completed successfully!")
    
    # Plot I-V characteristics
    iv_data = sim_complete.get_iv_data()
    V_sim, I_sim = iv_data.get_iv_data(electrode=2)
    
    plt.figure(figsize=(12, 5))
    
    # Forward bias - linear
    plt.subplot(1, 2, 1)
    mask_fwd = V_sim >= 0
    plt.plot(V_sim[mask_fwd], np.abs(I_sim[mask_fwd]) * 1e6, 'b-', linewidth=2)
    plt.xlabel('Voltage (V)', fontsize=12)
    plt.ylabel('Current (µA)', fontsize=12)
    plt.title('Forward Bias I-V (Linear)', fontsize=14)
    plt.grid(True, alpha=0.3)
    
    # Full range - log
    plt.subplot(1, 2, 2)
    mask = np.abs(I_sim) > 1e-20
    plt.semilogy(V_sim[mask], np.abs(I_sim[mask]), 'b-', linewidth=2)
    plt.xlabel('Voltage (V)', fontsize=12)
    plt.ylabel('|Current| (A)', fontsize=12)
    plt.title('Full I-V (Log Scale)', fontsize=14)
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Plot equilibrium band diagram
    print("\nEquilibrium Band Diagram:")
    sim_complete.plot_band_diagram(title="Complete Schottky Diode - Equilibrium")
else:
    print(f"Simulation failed with return code: {result_complete.returncode}")
    print("\nGenerated PADRE Input Deck:")
    print(sim_complete.generate_deck())

---

## 8. Exercises

### Exercise 1: Metal Selection
Create Schottky diodes with Ti, Au, and Pt contacts. Compare their I-V characteristics.

### Exercise 2: Doping Effect
How does substrate doping affect the depletion width and capacitance?

### Exercise 3: Temperature Dependence
The saturation current varies as $T^2 \exp(-q\phi_B/kT)$. Simulate at different temperatures.

In [ ]:
# Exercise 1: Different metals - Run actual simulations
metal_data = [
    ('Ti', 4.33),
    ('Au', 5.10),
    ('Pt', 5.65)
]

print("Metal Comparison - Running PADRE Simulations:")
print("="*60)

metal_simulations = {}
for metal, phi_m in metal_data:
    phi_B = phi_m - chi_si
    Js = A_star * T**2 * np.exp(-phi_B / kT)
    
    sim = create_schottky_diode(
        doping=1e17,
        workfunction=phi_m,
        log_iv=True,
        iv_file=f"{metal.lower()}_iv",
        forward_sweep=(0.0, 0.6, 0.02)
    )
    
    print(f"\n{metal}:")
    print(f"  Workfunction: {phi_m:.2f} eV")
    print(f"  Barrier height: {phi_B:.2f} eV")
    print(f"  Theoretical Js: {Js:.2e} A/cm^2")
    
    print(f"  Running simulation...")
    result = sim.run()
    if result.returncode == 0:
        print(f"  Completed!")
        metal_simulations[metal] = sim
    else:
        print(f"  Failed!")

# Plot comparison
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
colors = {'Ti': 'blue', 'Au': 'gold', 'Pt': 'gray'}
for metal, sim in metal_simulations.items():
    iv_data = sim.get_iv_data()
    V, I = iv_data.get_iv_data(electrode=2)
    plt.plot(V, np.abs(I) * 1e6, '-', color=colors[metal], linewidth=2, label=metal)

plt.xlabel('Voltage (V)', fontsize=12)
plt.ylabel('Current (µA)', fontsize=12)
plt.title('Metal Contact Comparison (Linear)', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
for metal, sim in metal_simulations.items():
    iv_data = sim.get_iv_data()
    V, I = iv_data.get_iv_data(electrode=2)
    mask = np.abs(I) > 1e-20
    plt.semilogy(V[mask], np.abs(I[mask]), '-', color=colors[metal], linewidth=2, label=metal)

plt.xlabel('Voltage (V)', fontsize=12)
plt.ylabel('Current (A)', fontsize=12)
plt.title('Metal Contact Comparison (Log)', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Find turn-on voltages
print("\nTurn-on Voltages (at 1µA) from PADRE:")
print("-"*40)
for metal, sim in metal_simulations.items():
    iv_data = sim.get_iv_data()
    V, I = iv_data.get_iv_data(electrode=2)
    try:
        V_on = np.interp(1e-6, np.abs(I), V)
        print(f"  {metal}: {V_on:.3f} V")
    except:
        print(f"  {metal}: Could not extract")

---

## Summary

In this notebook, you learned:

1. **Schottky Barrier Formation**: Metal workfunction determines barrier height
2. **Thermionic Emission**: Current mechanism in Schottky diodes
3. **Advantages over PN**: Lower turn-on, faster switching
4. **Band Diagrams**: How bias affects the band structure
5. **Barrier Lowering**: Image force effect at high fields

**Key Equations:**
- Barrier height: $\phi_B = \phi_m - \chi$
- Thermionic current: $J = A^*T^2 e^{-q\phi_B/kT}(e^{qV/kT} - 1)$

**Next**: [04 - MOSFET](04_MOSFET.ipynb) - Field-effect transistors